In [1]:
import os
import requests
import json
from dotenv import load_dotenv
from openai import OpenAI

In [2]:
load_dotenv()
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
AIDEVS_API_KEY = os.environ["AIDEVS_API_KEY"]

In [3]:
client = OpenAI()

In [4]:
response = client.moderations.create(input="To jest zdanie, które powinno przejść")
output = response.results[0]

In [5]:
print(output.category_scores.__dict__, end="\n\n")
print(output.categories.__dict__, end="\n\n")
print(output.flagged)

{'harassment': 0.00035014841705560684, 'harassment_threatening': 5.864134800503962e-05, 'hate': 0.0005976696847938001, 'hate_threatening': 8.453023838228546e-06, 'self_harm': 2.443450057398877e-06, 'self_harm_instructions': 1.6457053675367206e-07, 'self_harm_intent': 5.065106734036817e-07, 'sexual': 0.0027317798230797052, 'sexual_minors': 0.000356318021658808, 'violence': 0.0013797468272969127, 'violence_graphic': 2.5857763830572367e-05}

{'harassment': False, 'harassment_threatening': False, 'hate': False, 'hate_threatening': False, 'self_harm': False, 'self_harm_instructions': False, 'self_harm_intent': False, 'sexual': False, 'sexual_minors': False, 'violence': False, 'violence_graphic': False}

False


In [6]:
response = requests.post("https://tasks.aidevs.pl/token/moderation", json=dict(apikey=AIDEVS_API_KEY))
response_json = json.loads(response.text)
token = response_json["token"]

response = requests.get(f"https://tasks.aidevs.pl/task/{token}")
response_json = json.loads(response.text)

print(response_json)

{'code': 0, 'msg': 'please return 0/1 for each sentence to classify if this is content that requires moderation', 'input': ['ten gość musi zginąć. Nie pozwole sobię na obrażanie mnie.', 'majonez Winiary jest lepszy od Kieleckiego', 'Sasha.Grey.s3x.p0rn.extreme-interracial.S03E12.DVDRip.mp4', 'azjaci są głupi i brzydcy i nie powinni żyć']}


In [8]:
flag_list = []
for msg in response_json["input"]:
    print(f"Checking sentence: {msg}")
    res = client.moderations.create(input=msg)
    otp = res.results[0]
    flag_list.append(int(otp.flagged))
    
print(f"Flagged sentences: {flag_list}")

Checking sentence: ten gość musi zginąć. Nie pozwole sobię na obrażanie mnie.
Checking sentence: majonez Winiary jest lepszy od Kieleckiego
Checking sentence: Sasha.Grey.s3x.p0rn.extreme-interracial.S03E12.DVDRip.mp4
Checking sentence: azjaci są głupi i brzydcy i nie powinni żyć
Flagged sentences: [1, 0, 1, 1]


In [10]:
response = requests.post(f"https://tasks.aidevs.pl/answer/{token}", json=dict(answer=flag_list))
print(response.text)

{
    "code": 0,
    "msg": "OK",
    "note": "CORRECT"
}
